In [1]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import sys
sys.path.append("/home/cat/code/widefield/") # Adds higher directory to python modules path.



import numpy as np
import os

# pre process data module
from ProcessCalcium import ProcessCalcium

# 
from scipy.signal import hilbert, chirp

Autosaving every 180 seconds


In [3]:
##########################################################################
##########################################################################
##########################################################################
from scipy.signal import butter, lfilter, filtfilt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def butter_lowpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = butter(order, normal_cutoff, btype='lowpass')
    return b, a

def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=4):
    # Source: https://github.com/guillaume-chevalier/filtering-stft-and-laplace-transform
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = filtfilt(b, a, data)
    return y



fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar3_30Hz/IJ1pm_Mar3_30Hz_locanmf.npz'

data = np.load(fname, allow_pickle=True)

trials = data['temporal_trial']
print (trials.shape)
random = data['temporal_random']


names = data['names']
print (names)

name = 'motor'

t = []
r = []
for k in range(trials.shape[1]):
    if name in names[k]:
        print (k)
        t.append(trials[:,k].mean(0))
        r.append(random[:,k].mean(0))
        
# 
t = np.array(t).mean(0)
r = np.array(r).mean(0)
print (t.shape)

#
#t -= np.mean(t)
#r -= np.mean(r)

filter_cutoff = 14
t = butter_lowpass_filter(t, filter_cutoff,30)*100

x = np.arange(t.shape[0])/30.-30

analytic_signal = hilbert(t)
amplitude_envelope = np.abs(analytic_signal)
amplitude_envelope = butter_lowpass_filter(amplitude_envelope,.5,30)

#analytical_signal = hilbert(sensor)


#####################################################
#####################################################
#####################################################
fig=plt.figure()

#plt.plot(x,analytic_signal.real, c='black')
#plt.plot(x,analytic_signal.imag, c='green')

start = 300
end = 1050
t = t[start:end]
r = r[start:end]
x = x[start:end]
amplitude_envelope = amplitude_envelope[start:end]

plt.plot(x,t, c='blue', linewidth=3)
plt.plot(x, 
         amplitude_envelope, '--', c='blue', linewidth=3)


# 
plt.plot([x[0], x[-1]],[0,0],c='black', linewidth=3, label='neural')
plt.plot([x[600], x[600]],[-5,7],'--',c='black', linewidth=3, label='Envelope')

plt.xlim(x[0],x[-1])
plt.ylim(-5,7)
# plt.plot(x, 
#          -amplitude_envelope, c='blue')
#plt.plot(x,r)
plt.legend()

if False:
    plt.savefig('/home/cat/envelope.svg')
    plt.close()
else:
    plt.show()

(132, 16, 1801)
['Retrosplenial area, dorsal part, layer 1 - right'
 'Retrosplenial area, lateral agranular part, layer 1 - right'
 'Primary visual area, layer 1 - right'
 'Primary somatosensory area, upper limb, layer 1 - right'
 'Primary somatosensory area, lower limb, layer 1 - right'
 'Primary somatosensory area, barrel field, layer 1 - right'
 'Secondary motor area, layer 1 - right'
 'Primary motor area, Layer 1 - right'
 'Primary motor area, Layer 1 - left'
 'Secondary motor area, layer 1 - left'
 'Primary somatosensory area, barrel field, layer 1 - left'
 'Primary somatosensory area, lower limb, layer 1 - left'
 'Primary somatosensory area, upper limb, layer 1 - left'
 'Primary visual area, layer 1 - left'
 'Retrosplenial area, lateral agranular part, layer 1 - left'
 'Retrosplenial area, dorsal part, layer 1 - left']
6
7
8
9
(1801,)
